NOTE: This is a trial file which i used to test the cleaning i did in pandas before taking across to data_cleaning.py

In [2]:
import pandas as pd
import yaml
import tabula
import re
import numpy as np
import boto3
from sqlalchemy import create_engine, inspect
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning


In [5]:
db = DatabaseConnector() 

users = DataExtractor.read_rds_table(db, 'legacy_users')

['legacy_store_details', 'legacy_users', 'orders_table']


First Check and Convert All types

In [6]:
users.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

In [80]:
users['first_name'] = users['first_name'].astype('string')
users['last_name'] = users['last_name'].astype('string')
users['company'] = users['company'].astype('string')
users['email_address'] = users['email_address'].astype('string')
users['address'] = users['address'].astype('string')
users['country'] = users['country'].astype('category')
users['country_code'] = users['country_code'].astype('category')
users['phone_number'] = users['phone_number'].astype('string')
users['user_uuid'] = users['user_uuid'].astype('string')


Check again to make sure changes occurred

In [81]:
users.dtypes

index               int64
first_name         string
last_name          string
date_of_birth      object
company            string
email_address      string
address            string
country          category
country_code     category
phone_number       string
join_date          object
user_uuid          string
dtype: object

set index as index, as there are two columns for this usage

In [82]:
users = users.set_index('index')

check the categorical country code variables and remove all wrong values, convert any mistakes

In [83]:
users['country_code'].value_counts()

GB            9365
DE            4708
US            1205
NULL            21
GGB              6
PG8MOC0UZI       1
XKI9UXSCZ1       1
VSM4IZ4EL3       1
RVRFD92E48       1
QVUW9JSKY3       1
QREF9WLI2A       1
0CU6LW3NKB       1
OS2P9CMHR6       1
44YAIDY048       1
NTCGYW8LVC       1
LZGTB0T5Z7       1
IM8MN1L9MJ       1
FB13AKRI21       1
5D74J6FPFJ       1
XPVCZE2L8B       1
Name: country_code, dtype: int64

In [84]:
users = users[users['country_code'].str.len() < 4]
users['country_code'] = users['country_code'].replace('GGB', 'GB')

check country code again

In [85]:
users['country_code'].value_counts()

GB            9371
DE            4708
US            1205
0CU6LW3NKB       0
PG8MOC0UZI       0
XKI9UXSCZ1       0
VSM4IZ4EL3       0
RVRFD92E48       0
QVUW9JSKY3       0
QREF9WLI2A       0
NULL             0
OS2P9CMHR6       0
44YAIDY048       0
NTCGYW8LVC       0
LZGTB0T5Z7       0
IM8MN1L9MJ       0
FB13AKRI21       0
5D74J6FPFJ       0
XPVCZE2L8B       0
Name: country_code, dtype: int64

standardise the phone numbers and remove all special characters

In [86]:
users['phone_number']

index
0         +49(0) 047905356
1          (0161) 496 0674
2        +44(0)121 4960340
3          (0306) 999 0871
4            0121 496 0225
               ...        
14913      +44(0)292018946
14994     +44(0)1144960977
15012          02984 08192
15269         239.711.3836
1249      +44(0)1314960870
Name: phone_number, Length: 15284, dtype: string

In [87]:
def standardise_phone_number(phone_number):
    phone_number = phone_number.replace(" ", "")
    phone_number = phone_number.replace("(", "")
    phone_number = phone_number.replace(")", "")
    phone_number = phone_number.replace(".", "")
    phone_number = phone_number.replace("-", "")
    phone_number = phone_number.replace("x", "")
    phone_number = list(phone_number)
    if phone_number[0] == '+':
        phone_number = phone_number[3:]
    phone_number = ''.join(phone_number)
    return phone_number

users['phone_number'] = users['phone_number'].apply(standardise_phone_number)
users['phone_number'] = users['phone_number'].astype('string')
users['phone_number']

index
0         0047905356
1        01614960674
2        01214960340
3        03069990871
4        01214960225
            ...     
14913     0292018946
14994    01144960977
15012     0298408192
15269     2397113836
1249     01314960870
Name: phone_number, Length: 15284, dtype: string

remove convert like "1968 October 16" and convert date_of_birth/join_date to a datetime variable

In [88]:
users["date_of_birth"].str.len().max()

16

In [89]:
def standardise_date(date):
    
    if len(date) == 10:
        return date
    
    months_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 
                   'May':'05', 'June':'06', 'July':'07', 'August':'08',
                   'September':'09', 'October':'10', 'November':'11', 'December':'12'}
    
    split_date = date.split(" ")
    day = split_date[-1]
    if split_date[0][0].isalpha():
        month = split_date[0]
        year = split_date[1]
    else:
        month = split_date[1]
        year = split_date[0]
    month = months_dict[month]
    # print(day)
    # print(month)
    # print(year)
    return f"{year}-{month}-{day}"

In [90]:
users['date_of_birth'] = users['date_of_birth'].apply(standardise_date)

In [91]:
users["date_of_birth"] = pd.to_datetime(users["date_of_birth"], format="%Y-%m-%d")
users["date_of_birth"]

index
0       1990-09-30
1       1940-12-01
2       1995-08-02
3       1972-09-23
4       1952-12-20
           ...    
14913   1943-08-09
14994   1948-08-20
15012   1940-10-09
15269   1952-06-04
1249    1994-03-27
Name: date_of_birth, Length: 15284, dtype: datetime64[ns]

In [92]:
users["join_date"].str.len().max()

17

In [93]:
users['join_date'] = users['join_date'].apply(standardise_date)

In [94]:
users["join_date"] = pd.to_datetime(users["join_date"], format="%Y-%m-%d")
users["join_date"]

index
0       2018-10-10
1       2001-12-20
2       2016-12-16
3       2004-02-23
4       2006-09-01
           ...    
14913   2016-04-15
14994   2020-07-20
15012   2021-03-07
15269   2011-01-03
1249    2015-08-28
Name: join_date, Length: 15284, dtype: datetime64[ns]

check and remove users with join dates before their birth dates

In [95]:
users[users["join_date"] < users["date_of_birth"]].info

<bound method DataFrame.info of       first_name  last_name date_of_birth                          company  \
index                                                                        
39          Paul   Misicher    2006-08-24                             Kuhl   
64         Tracy     Turner    2003-11-05        Higgins, Graham and Singh   
100      William       Wood    1994-05-08      Barker, Patel and Stevenson   
102         Alan       Hall    2000-03-07  Richardson, Hayward and Farrell   
127        Wendy      Burke    2005-01-25                   Nixon and Sons   
...          ...        ...           ...                              ...   
15286     Thomas      Begum    1996-12-16     Phillips, Chandler and Evans   
15287      Julie      Lewis    2001-02-20                   Smith and Sons   
15314   Marliese  Holzapfel    2004-01-11                          Ruppert   
10454    Stephen   Williams    2005-05-06        Lewis, Fletcher and Allan   
12523    Stephen   Alvarado    2

In [96]:
users = users[users["join_date"] > users["date_of_birth"]]

In [23]:
users.to_string('filename.txt')

In [74]:
users.duplicated(['email_address']).sum()


0

In [97]:
users.dtypes

first_name               string
last_name                string
date_of_birth    datetime64[ns]
company                  string
email_address            string
address                  string
country                category
country_code           category
phone_number             string
join_date        datetime64[ns]
user_uuid                string
dtype: object

In [58]:
dim_card_details = DatabaseConnector.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

In [56]:
dim_card_details = DataCleaning.clean_card_data(dim_card_details)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\steph\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\steph\AppData\Local\Temp\ipykernel_10016\4185783719.py", line 1, in <module>
    dim_card_details = DataCleaning.clean_card_data(dim_card_details)
AttributeError: type object 'DataCleaning' has no attribute 'clean_card_data'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\steph\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\steph\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\steph\AppData\Roaming\Python\Python310\site-packages\IPytho

In [54]:
len(dim_card_details)

15284

In [65]:
type(dim_card_details)

pandas.core.frame.DataFrame

In [62]:
dim_card_details.dtypes


card_number               string
expiry_date               object
card_provider             string
date_payment_confirmed    object
dtype: object

In [61]:
dim_card_details = dim_card_details[dim_card_details['expiry_date'].str.len() == 5]

In [59]:
dim_card_details['card_number'] = dim_card_details['card_number'].astype('string')

In [60]:
dim_card_details['card_provider'] = dim_card_details['card_provider'].astype('string')

In [48]:
outliers = dim_card_details[dim_card_details['date_payment_confirmed'].str.len() < 10]
outliers

,card_number,expiry_date,card_provider,date_payment_confirmed


In [63]:
def standardise_date(date):
    
    if len(date) == 10:
        return date
    
    months_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 
                   'May':'05', 'June':'06', 'July':'07', 'August':'08',
                   'September':'09', 'October':'10', 'November':'11', 'December':'12'}
    
    split_date = date.split(" ")
    day = split_date[-1]
    if split_date[0][0].isalpha():
        month = split_date[0]
        year = split_date[1]
    else:
        month = split_date[1]
        year = split_date[0]
    month = months_dict[month]
    # print(day)
    # print(month)
    # print(year)
    return f"{year}-{month}-{day}"

In [64]:
dim_card_details['date_payment_confirmed'] = dim_card_details['date_payment_confirmed'].apply(standardise_date)

In [66]:
dim_card_details['date_payment_confirmed'] = pd.to_datetime(dim_card_details['date_payment_confirmed'], format="%Y-%m-%d")
dim_card_details['date_payment_confirmed']

0       2015-11-25
1       2001-06-18
2       2000-12-26
3       2011-02-12
4       1997-03-13
           ...    
15304   1997-06-06
15305   2004-06-16
15306   2020-02-05
15307   2008-06-16
15308   2009-02-04
Name: date_payment_confirmed, Length: 15284, dtype: datetime64[ns]

In [67]:
dim_card_details['expiry_date']

0        09/26
1        10/23
2        06/23
3        09/27
4        10/25
         ...  
15304    12/28
15305    11/24
15306    04/24
15307    06/27
15308    02/30
Name: expiry_date, Length: 15284, dtype: object

In [68]:
dim_card_details['expiry_date'] = pd.to_datetime(dim_card_details['expiry_date'], format="%m/%y")

In [258]:
with open('api_key.yaml', 'r') as file:
    key_dict = yaml.load(file, Loader=yaml.FullLoader)

In [259]:
api_db = DataExtractor.retrieve_stores_data(f'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/', key_dict)

In [260]:
api_db

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,"Studio 6\nStephen landing\nSouth Simon\nB77 2WA, Belper",53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High Wycombe",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe


In [261]:
api_db = api_db.set_index('index')

In [262]:
api_db['country_code'] = api_db['country_code'].astype('category')
api_db['continent'] = api_db['continent'].astype('category')
api_db['store_type'] = api_db['store_type'].astype('category')
api_db['locality'] = api_db['locality'].astype('string')
api_db['store_code'] = api_db['store_code'].astype('string')
api_db = api_db.drop('lat', axis=1)

In [263]:
api_db['country_code'].value_counts()


GB            266
DE            141
US             34
NULL            3
B3EH2ZGQAV      1
F3AO8V2LHU      1
FP8DLXQVGH      1
HMHIFNLOBN      1
OH20I92LX3      1
OYVW925ZL8      1
YELVM536YT      1
Name: country_code, dtype: int64

In [264]:
bad = api_db[api_db['country_code'].str.len() >=4]
bad

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
63,6FWDZHD7PW,1ZVU03X2P6,9IBH8Y4Z0S,NRQKZWJ9OZ,BIP8K8JJW2,ZCXWWKF45G,QP74AHEQT0,1CJ5OAU4BR,YELVM536YT,QMAVR5H3LD
172,UBCIFQLSNY,Q1TJY8H1ZH,1T6B406CI8,QIUU9SVP51,SKBXAXF5G5,7AHXLXIUEF,O0QJIRC943,3ZR3F89D97,FP8DLXQVGH,LU3E036ZD9
217,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
231,RC99UKMZB2,2YBZ1440V6,6LVWPU1G64,Y8J0Z2W8O9,2429OB3LMM,0OLAK2I6NS,50IB01SFAZ,L13EQEQODP,HMHIFNLOBN,5586JCLARW
333,X349GIDWKU,O7NF1FZ74Y,RX9TCP2RGB,ISEE8A57FE,74BY7HSB6P,A3PMVM800J,0RSNUU3DF5,J3BPB68Z1J,F3AO8V2LHU,GFJQ2AAEQ8
381,ZBGB54ID4H,SKO4NMRNNF,CQMHKI78BX,T0R2CQBDUS,GT1FO6YGD4,GMMB02LA9V,B4KVQB3P5Y,AJHOMDOHZ4,OH20I92LX3,SLQBD982C0
405,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
414,XTUAV57DP4,ID819KG3X5,RY6K0AUE7F,TUOKF5HAAQ,FRTGHAA34B,13PIY8GD1H,X0FE7E2EOG,AE7EEW4HSS,OYVW925ZL8,XQ953VS0FG
437,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [265]:
api_db = api_db[api_db['country_code'].str.len() < 4]

In [266]:
api_db['continent'].value_counts()

Europe        383
America        32
eeEurope       24
eeAmerica       2
1WZB1TE1HL      0
5586JCLARW      0
GFJQ2AAEQ8      0
LU3E036ZD9      0
NULL            0
QMAVR5H3LD      0
SLQBD982C0      0
XQ953VS0FG      0
Name: continent, dtype: int64

In [267]:
def correct_continent(input):
    if len(input) <= 7:
        return input
    return input[2:]

In [268]:
api_db['continent'] = api_db['continent'].apply(correct_continent)

In [269]:
api_db['continent'] = api_db['continent'].astype('category')

In [218]:
bad = api_db[api_db['opening_date'].str.len() > 10]
bad

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
10,"8 Gareth skyway\nSimmonsview\nSW4 7PL, Rutherglen",55.82885,Rutherglen,RU-9F1136B4,32,October 2012 08,Local,-4.21376,GB,Europe
11,"Bergerstr. 1/3\n59183 Seelow, Siegburg",50.80019,Siegburg,SI-ECD52CD9,25,July 2015 14,Local,7.20769,DE,Europe
122,"Studio 66K\nChloe road\nStanleyberg\nS0 6NE, Selby",53.78333,Selby,SE-8DF6557B,20,2020 February 01,Local,-1.06667,GB,Europe
143,"Eugenie-Pruschke-Straße 5\n31111 Badibling, Memmingen",47.98372,Memmingen,ME-9940FF73,7,May 2003 27,Mall Kiosk,10.18527,DE,Europe
190,"70 Ward center\nPort Nicola\nDT73 4WY, Bushey",51.64316,Bushey,BU-3FC2A064,25,2016 November 25,Local,-0.36053,GB,Europe
242,"Flat 84d\nCarol points\nPaigeland\nG3 0PD, Clacton-on-Sea",51.78967,Clacton-on-Sea,CL-DA786EF6,22,October 2006 04,Local,1.15597,GB,Europe
292,"Löwerallee 49\n00561 Miltenberg, Landshut",48.52961,Landshut,LA-55069B69,6,2001 May 04,Mall Kiosk,12.16179,DE,Europe
340,"148 Samuel course\nNicolaside\nGY3E 5SD, Great Wyrley",52.66277,Great Wyrley,GR-250FBD0C,27,1994 November 24,Local,-2.01111,GB,Europe
369,"0 Marian circles\nPort Kayleigh\nPR5 6LW, Bushey",51.64316,Bushey,BU-B5B6F452,25,February 2009 28,Local,-0.36053,GB,Europe


In [271]:
def standardise_date(date):
    
    if len(date) == 10:
        return date
    
    months_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 
                   'May':'05', 'June':'06', 'July':'07', 'August':'08',
                   'September':'09', 'October':'10', 'November':'11', 'December':'12'}
    
    split_date = date.split(" ")
    day = split_date[-1]
    if split_date[0][0].isalpha():
        month = split_date[0]
        year = split_date[1]
    else:
        month = split_date[1]
        year = split_date[0]
    month = months_dict[month]
    # print(day)
    # print(month)
    # print(year)
    return f"{year}-{month}-{day}"

In [272]:
api_db['opening_date'] = api_db['opening_date'].apply(standardise_date)

In [273]:
api_db['opening_date'] = pd.to_datetime(api_db['opening_date'], format="%Y-%m-%d")

In [274]:
def standardise_storenumbers(numb):
    
    numb = ''.join(c for c in numb if c.isdigit())
    return numb

In [276]:
api_db['staff_numbers'] = api_db['staff_numbers'].apply(standardise_storenumbers)

In [277]:
api_db['staff_numbers'] = api_db['staff_numbers'].astype('int64')

In [278]:
api_db

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,"Studio 6\nStephen landing\nSouth Simon\nB77 2WA, Belper",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...
445,"Flat 7\nStephanie lake\nMorrisside\nHP8 8LH, Cowes",50.76306,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High Wycombe",51.62907,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe


In [279]:
pd.set_option('display.max_colwidth', None)

In [228]:
api_db

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, High Wycombe",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,"Studio 6\nStephen landing\nSouth Simon\nB77 2WA, Belper",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...
445,"Flat 7\nStephanie lake\nMorrisside\nHP8 8LH, Cowes",50.76306,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High Wycombe",51.62907,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe


In [280]:
def standardise_address(addr):
    if len(addr) == 3:
        return addr
    addr = re.split('\n|,', addr)
    addr = addr[:-1]
    addr = ', '.join(addr)
    return addr

In [281]:
api_db['address'] = api_db['address'].apply(standardise_address)


In [282]:
api_db['address']

index
0                                                  N/A
1         Flat 72W, Sally isle, East Deantown, E7B 8EB
2                    Heckerstraße 4/5, 50491 Säckingen
3              5 Harrison tunnel, South Lydia, WC9 2BE
4      Studio 6, Stephen landing, South Simon, B77 2WA
                            ...                       
445        Flat 7, Stephanie lake, Morrisside, HP8 8LH
446                  Täschestraße 25, 39039 Nördlingen
448           Studio 8, Moss mall, West Linda, M0E 6XR
449                        Baumplatz 6, 80114 Kötzting
450       Gotthilf-Rose-Straße 7/3, 45457 Feuchtwangen
Name: address, Length: 441, dtype: object

In [283]:
api_db = api_db.replace('N/A', np.NaN)

In [288]:
api_db.loc[0,['latitude']]=np.NaN

In [289]:
api_db

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,NaN,NaN,<NA>,WEB-1388012W,325,2010-06-12,Web Portal,NaN,GB,Europe
1,"Flat 72W, Sally isle, East Deantown, E7B 8EB",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5, 50491 Säckingen",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel, South Lydia, WC9 2BE",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,"Studio 6, Stephen landing, South Simon, B77 2WA",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...
445,"Flat 7, Stephanie lake, Morrisside, HP8 8LH",50.76306,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,"Täschestraße 25, 39039 Nördlingen",52.2,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
448,"Studio 8, Moss mall, West Linda, M0E 6XR",51.62907,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe


In [290]:
api_db['longitude']

index
0           NaN
1      51.62907
2      48.52961
3         51.26
4       53.0233
         ...   
445    50.76306
446        52.2
448    51.62907
449    49.03685
450    50.64336
Name: longitude, Length: 441, dtype: object

In [238]:
api_db.dtypes

address                  object
longitude                object
locality                 string
store_code               string
staff_numbers             int64
opening_date     datetime64[ns]
store_type             category
latitude                 object
country_code           category
continent              category
dtype: object

In [291]:
def standardise_longitude(val):
    return round(float(val), 2) if val != 'NaN' else 'NaN'

In [292]:
api_db['longitude'] = api_db['longitude'].apply(standardise_longitude)


In [293]:
api_db['latitude'] = api_db['latitude'].apply(standardise_longitude)

In [294]:
api_db['latitude'].describe()

count    440.000000
mean      -4.729159
std       25.361253
min     -122.390000
25%       -2.520000
50%       -0.750000
75%        8.467500
max       13.320000
Name: latitude, dtype: float64

In [295]:
api_db['address'] = api_db['address'].astype('string')



In [296]:
api_db

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
index,,,,,,,,,,
0,<NA>,NaN,<NA>,WEB-1388012W,325,2010-06-12,Web Portal,NaN,GB,Europe
1,"Flat 72W, Sally isle, East Deantown, E7B 8EB",51.63,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.75,GB,Europe
2,"Heckerstraße 4/5, 50491 Säckingen",48.53,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16,DE,Europe
3,"5 Harrison tunnel, South Lydia, WC9 2BE",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.19,GB,Europe
4,"Studio 6, Stephen landing, South Simon, B77 2WA",53.02,Belper,BE-18074576,35,2019-09-09,Local,-1.48,GB,Europe
...,...,...,...,...,...,...,...,...,...,...
445,"Flat 7, Stephanie lake, Morrisside, HP8 8LH",50.76,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.30,GB,Europe
446,"Täschestraße 25, 39039 Nördlingen",52.20,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63,DE,Europe
448,"Studio 8, Moss mall, West Linda, M0E 6XR",51.63,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.75,GB,Europe


In [2]:
import boto3

s3 = boto3.client('s3')

In [3]:
s3.download_file('data-handling-public', 'products.csv', 's3_product_data.csv')

In [4]:
df = pd.read_csv('s3_product_data.csv')

In [5]:
df.dtypes

Unnamed: 0        int64
product_name     object
product_price    object
weight           object
category         object
EAN              object
date_added       object
uuid             object
removed          object
product_code     object
dtype: object

In [6]:
df

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [7]:
df = df[df['weight'].str.len() < 10]
df

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [8]:
def standardise_weight(weight):
    if weight == 'nan':
        return weight
    if 'kg' in weight:
        multiplier = 1
    elif 'k' not in weight:
        if 'oz' not in weight:
            multiplier = 1000
        else:
            multiplier = 35.274
    output = ''
    for letter in weight:
        if letter.isnumeric() or letter == '.':
            output += letter
        elif letter == 'x':
            multiplier = multiplier / float(output)
            output = ''
        elif letter.isalpha():
            return round(float(output)/multiplier, 3)
            

In [9]:
df['weight'] = df['weight'].apply(standardise_weight)

C:\Users\steph\AppData\Local\Temp\ipykernel_13968\1189199909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight'] = df['weight'].apply(standardise_weight)


In [10]:
df

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.600,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.480,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,0.590,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,0.540,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.910,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,0.125,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,0.134,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,0.820,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,0.125,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [18]:
df.dtypes

product_name      object
product_price    float64
weight           float64
category          object
EAN               object
date_added        object
uuid              object
removed           object
product_code      object
dtype: object

In [12]:
df.rename(columns = {'Unnamed: 0':'index'}, inplace = True)

C:\Users\steph\AppData\Local\Temp\ipykernel_13968\1294815572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'Unnamed: 0':'index'}, inplace = True)


In [13]:
df = df.set_index('index')

In [14]:
df

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
index,,,,,,,,,
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.600,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.480,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,0.590,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,0.540,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.910,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...
1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,0.125,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,0.134,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,RAC 12V Wet & Dry Vacuum,£18.00,0.820,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b


In [15]:
df['product_price'] = df['product_price'].astype('string')

In [16]:
def standardise_prod_price(price):
    price = price[1:]
    return float(price)

In [17]:
df['product_price'] = df['product_price'].apply(standardise_prod_price)

In [269]:
df['product_price'] = df['product_price'].astype('float')

In [19]:
df['category'] = df['category'].astype('category')

In [20]:
df['EAN'] = df['EAN'].astype('string')

In [272]:
bad = df[df['date_added'].str.len() > 10]
bad

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
index,,,,,,,,,
307,Pets Pantry Complete Meaty Chunks Beef 2.75kg,3.69,2.75,pets,4525436759460,2018 October 22,6fc67918-7940-4b1f-bba4-47f3e6dfc578,Still_avaliable,l1-2836416D
1217,Black Ceramic Vase,8.00,0.65,homeware,5806423927017,September 2017 06,38f124ff-1d42-4359-96f0-91490c9a6f26,Still_avaliable,M6-7203684r


In [273]:
def standardise_date(date):
    
    if len(date) == 10:
        return date
    
    months_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 
                   'May':'05', 'June':'06', 'July':'07', 'August':'08',
                   'September':'09', 'October':'10', 'November':'11', 'December':'12'}
    
    split_date = date.split(" ")
    day = split_date[-1]
    if split_date[0][0].isalpha():
        month = split_date[0]
        year = split_date[1]
    else:
        month = split_date[1]
        year = split_date[0]
    month = months_dict[month]
    # print(day)
    # print(month)
    # print(year)
    return f"{year}-{month}-{day}"

In [274]:
df['date_added'] = df['date_added'].apply(standardise_date)

In [275]:
df['date_added'] = pd.to_datetime(df['date_added'], format="%Y-%m-%d")

In [276]:
df['uuid'] = df['uuid'].astype('string')

In [277]:
df['removed'] = df['removed'].astype('category')
df['removed'].value_counts()

Still_avaliable    1752
Removed              94
Name: removed, dtype: int64

In [278]:
df['product_code'] = df['product_code'].astype('string')

In [281]:
df['product_name'] = df['product_name'].astype('string')

In [282]:
df.dtypes

product_name             string
product_price           float64
weight                  float64
category               category
EAN                      string
date_added       datetime64[ns]
uuid                     string
removed                category
product_code             string
dtype: object

In [1]:
import yaml

In [3]:
db = DatabaseConnector() 
orders = DataExtractor.read_rds_table(db, 'orders_table') 

['legacy_store_details', 'legacy_users', 'orders_table']


In [4]:
orders = orders.set_index('index')
orders

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
index,,,,,,,,,,
0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2
...,...,...,...,...,...,...,...,...,...,...
110548,110549,f0e8fff6-9998-4661-954b-0e258e09d33c,None,None,95c74b0a-d495-4359-b1c0-e2da511e8403,575421945446,KA-FA7ED3B8,C9-6827622o,NaN,4
82164,82164,1c80940a-d186-4ba9-9daa-8abd1aceae32,None,None,5d6fa6fe-e583-4baf-8bbb-d1dd6e2b551f,4971858637664481,WA-A41DA979,I0-1146408B,NaN,1
97599,97599,58598aca-049c-418e-8e39-46327634a7f1,Sharon,Miller,48b7f1fc-db13-4611-ad8e-3dac0b759488,4971858637664481,WEB-1388012W,A4-5443400b,NaN,4


In [5]:
orders = orders.sort_index()

In [6]:
orders

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
index,,,,,,,,,,
0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2
...,...,...,...,...,...,...,...,...,...,...
120118,120118,c2131886-7dc4-4812-af30-2182b3dcab68,None,None,f58cc2aa-e4b7-4893-a1e3-734c7e45c94e,2291071373356040,HI-F27C7A84,w5-6777421C,NaN,4
120119,120119,eca2b196-ec12-4853-b3d5-41e029d14774,None,None,2c4a5960-a6b1-4743-a74f-27c177d67b55,3599396269535600,DA-ACC520AE,z2-7184027O,NaN,3
120120,120120,c0b61063-d282-423f-84e3-81371263c980,None,None,44064842-6dbe-4c8d-8bb0-839df7d75d62,3562219981147900,TR-8CD56339,m4-507493i,NaN,4


In [7]:
orders = orders.drop(['level_0', 'first_name', 'last_name', '1'], axis = 1)

In [8]:
orders['date_uuid'] = orders['date_uuid'].astype('string')
orders['user_uuid'] = orders['user_uuid'].astype('string')
orders['store_code'] = orders['store_code'].astype('string')
orders['product_code'] = orders['product_code'].astype('string')


In [9]:
orders['card_number'] = orders['card_number'].astype('string')


In [137]:
link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'



In [160]:
events = pd.read_json(link)
events

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb


In [162]:
events = events[events['day'].str.len() <= 2]
events

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb


In [163]:
events['date_uuid'] = events['date_uuid'].astype('string')
events['time_period'] = events['time_period'].astype('category')
events['year'] = events['year'].astype('string')
events['month'] = events['month'].astype('string')
events['day'] = events['day'].astype('string')



In [164]:
events.dtypes

timestamp        object
month            string
year             string
day              string
time_period    category
date_uuid        string
dtype: object

In [142]:
#events['timestamp'] = pd.to_datetime(events['timestamp'], format='%H:%M:%S').dt.time

C:\Users\steph\AppData\Local\Temp\ipykernel_19988\1665738356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['timestamp'] = pd.to_datetime(events['timestamp'], format='%H:%M:%S').dt.time


In [165]:
def pad_day_month(day):
    if len(day) == 2:
        return day
    else: return '0' + day
events['day'] = events['day'].apply(pad_day_month)
events['month'] = events['month'].apply(pad_day_month)

In [166]:
events['date'] = events[['year', 'month', 'day']].agg('-'.join, axis=1)

In [167]:
events['iso'] = events[['date', 'timestamp']].agg(' '.join, axis=1)

In [168]:
events

,timestamp,month,year,day,time_period,date_uuid,date,iso
0,22:00:06,09,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad,2012-09-19,2012-09-19 22:00:06
1,22:44:06,02,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa,1997-02-10,1997-02-10 22:44:06
2,10:05:37,04,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31,1994-04-15,1994-04-15 10:05:37
3,17:29:27,11,2001,06,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8,2001-11-06,2001-11-06 17:29:27
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44,2015-12-31,2015-12-31 22:40:33
...,...,...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7,2022-11-12,2022-11-12 22:56:56
120157,18:25:20,05,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6,1997-05-31,1997-05-31 18:25:20
120158,18:21:40,09,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e,2011-09-13,2011-09-13 18:21:40
120159,19:10:53,07,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb,2013-07-12,2013-07-12 19:10:53


In [169]:
events['iso'] = pd.to_datetime(events['iso'])

In [170]:
events = events.drop(['date'], axis=1)

In [171]:
events.dtypes

timestamp              object
month                  object
year                   string
day                    object
time_period          category
date_uuid              string
iso            datetime64[ns]
dtype: object